# Criando bucket

Este trecho de código em Python usa a biblioteca boto3 para criar um bucket no Amazon S3, que é um serviço de armazenamento de objetos da AWS. A função create_s3_bucket recebe como parâmetros o nome do bucket e, opcionalmente, a região onde o bucket deve ser criado.

In [1]:
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError

def create_s3_bucket(bucket_name, region=None):
    try:
        # Inicializa a sessão e o cliente S3 usando o perfil configurado
        session = boto3.Session(profile_name="default")
        s3_client = session.client("s3", region_name=region)

        # Cria o bucket com a região especificada
        if region is None or region == "us-east-1":
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {"LocationConstraint": region}
            s3_client.create_bucket(
                Bucket=bucket_name, CreateBucketConfiguration=location
            )

        print(f"Bucket '{bucket_name}' criado com sucesso.")

    except NoCredentialsError:
        print("Credenciais não encontradas.")
    except PartialCredentialsError:
        print("Credenciais incompletas.")
    except ClientError as e:
        print(f"Ocorreu um erro ao tentar criar o bucket: {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

# Criando o bucket
bucket_name = "bucket-rodrigo-silva"
region = "us-east-1"
create_s3_bucket(bucket_name, region)


Bucket 'bucket-rodrigo-silva' criado com sucesso.


# Ler os Arquivos CSV e Fazer Upload para o Amazon S3

Este código Python lê dados de arquivos CSV e os faz upload para um bucket no Amazon S3. A função read_csv lê um arquivo CSV e retorna uma lista de dicionários, onde cada dicionário representa uma linha do CSV. A função upload_file_to_s3 inicializa uma sessão do boto3 usando o perfil padrão, cria um cliente S3, e faz o upload do arquivo especificado para o bucket S3 com uma estrutura de caminho que inclui a data de processamento. Exceções são tratadas para informar problemas relacionados a credenciais ou erros do cliente S3. Por fim, a função é chamada para fazer o upload de dois arquivos CSV (movies.csv e series.csv).




In [1]:
import os
import csv
import datetime
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError

def read_csv(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            data.append(row)
    return data

def upload_file_to_s3(bucket_name, file_path, object_name=None):
    try:
        session = boto3.Session(profile_name="default")
        s3_client = session.client("s3")

        if object_name is None:
            object_name = os.path.basename(file_path)

        # Obtendo a data de processamento para construir o caminho
        processing_date = datetime.datetime.now().strftime("%Y/%m/%d")
        # Construindo o caminho do arquivo no S3
        file_name = os.path.splitext(object_name)[0]  # Remove a extensão do arquivo
        s3_key = f"Raw/Local/CSV/{file_name}/{processing_date}/{object_name}"

        s3_client.upload_file(file_path, bucket_name, s3_key)

        print(f"Arquivo '{file_path}' enviado para o bucket '{bucket_name}' como '{s3_key}'.")

    except NoCredentialsError:
        print("Credenciais não encontradas.")
    except PartialCredentialsError:
        print("Credenciais incompletas.")
    except ClientError as e:
        print(f"Ocorreu um erro ao tentar enviar o arquivo: {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")

bucket_name = "bucket-rodrigo-silva"
region = "us-east-1"

# Upload dos arquivos
upload_file_to_s3(bucket_name, 'Data/movies.csv', 'movies.csv')
upload_file_to_s3(bucket_name, 'Data/series.csv', 'series.csv')


Arquivo 'Data/movies.csv' enviado para o bucket 'bucket-rodrigo-silva' como 'Raw/Local/CSV/movies/2024/05/28/movies.csv'.
Arquivo 'Data/series.csv' enviado para o bucket 'bucket-rodrigo-silva' como 'Raw/Local/CSV/series/2024/05/28/series.csv'.


# Construção do Arquivo 

Este Dockerfile cria um contêiner Docker para um aplicativo Python que interage com serviços da AWS usando a biblioteca boto3. Ele utiliza a imagem base python:3.9-slim para ser mais leve, copia todos os arquivos do diretório atual para o diretório /app no contêiner, define /app como o diretório de trabalho, instala boto3 sem usar cache para economizar espaço, e define como comando de entrada padrão a execução do script main.py com Python.

In [ ]:
# Use uma imagem base Python
FROM python:3.9-slim

# Copie os arquivos necessários para o contêiner
COPY . /app

# Defina o diretório de trabalho
WORKDIR /app

# Instale as dependências Python
RUN pip install --no-cache-dir boto3

# Comando de entrada padrão
CMD ["python", "main.py"]

# Crie o Volume

docker volume create FilmesSeries-vol

# Construa a Imagem

docker build -t data-lake-rodrigo .

# Execute o container com os volumes montados

docker run -v FilmesSeries-vol:/FilmesSeries -v C:/Users/Rodrigo/.aws:/root/.aws data-lake-rodrigo
